In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np
from datetime import datetime 

import csv
import requests

import time
import concurrent.futures

from pairs_selection.api_wrapper import alph_settings as alph, val_steps as val, alph_api_wrapper as wrap

In [3]:
settings = alph(
    apikey = "MD27QTTVAK3AJBUQ", 
    site = "https://www.alphavantage.co/query?", 
    date_range = [datetime(2020, 8, 9), datetime(2022, 2, 20)]
    )

In [4]:
tker_dict = val.get_tkers(settings)

In [5]:
valid_lst = val.get_valid_tkers(tker_dict)

Excluded: PANA, Panacea Acquisition Corp II - Class A
Excluded: RSPE, Invesco ESG S&P 500 Equal Weight ETF
Excluded: GAMCW, Golden Arrow Merger Corp - Warrants (31/07/2026)
Excluded: ZNOG, Zion Oil & Gas Inc
Excluded: QRTEP, Qurate Retail Inc
Excluded: ROCLW, Roth CH Acquisition V Co - Warrants (10/12/2026)
Excluded: PRM, Perimeter Solutions SA
Excluded: UPTD, TradeUP Acquisition Corp
Excluded: GTACW, Global Technology Acquisition Corp I - Warrants (19/10/2026)
Excluded: WWAC, Worldwide Webb Acquisition Corp - Class A
Excluded: GB-WS, Global Blue Group Holding AG - Warrants (28/08/2025)
Excluded: BRIVW, B Riley Principal 250 Merger Corp - Warrants (06/04/2026)
Excluded: QNGY, Quanergy Systems Inc
Excluded: FDUSZ, Fidus Investment Corp
Excluded: STMP, Stamps.com Inc
Excluded: LHDX, Lucira Health Inc
Excluded: HAYW, Hayward Holdings Inc
Excluded: AMAOW, American Acquisition Opportunity Inc - Warrants (17/03/2026)
Excluded: FORH, FORMIDABLE ETF 
Excluded: TINV-U, Tiga Acquisition Corp - U

In [16]:
test_df = pd.DataFrame(valid_lst)
display(test_df)
test_lst = test_df[0].tolist()
#print(test_lst)

,0,1,2,3,4,5,6
0,CYAN,Cyanotech Corp,NASDAQ,Stock,1993-06-22,null,Active
1,RMED,Ra Medical Systems Inc,NYSE MKT,Stock,2018-09-27,null,Active
2,CC,Chemours Company,NYSE,Stock,2015-07-01,null,Active
3,NVNOW,enVVeno Medical Corporation - Warrants (30/05/...,NASDAQ,Stock,2018-05-31,null,Active
4,ATEC,Alphatec Holdings Inc,NASDAQ,Stock,2006-06-02,null,Active
...,...,...,...,...,...,...,...
8051,KRA,Kraton Corp,NYSE,Stock,2009-12-17,null,Active
8052,EBR,Centrais Eletricas Brasileiras S.A.,NYSE,Stock,2008-11-19,null,Active
8053,MINM,Minim Inc,NASDAQ,Stock,2009-10-07,null,Active
8054,PAVE,Global X U.S. Infrastructure Development ETF,BATS,ETF,2017-03-08,null,Active


In [9]:
function = "TIME_SERIES_INTRADAY_EXTENDED"
ticker_lst = test_lst[:100]
interval = "1min"
slice = ["year1month1"] #[f"year1month{i}" for i in range(1, 4)] #first three months of intraday data
outputsize = "compact"
test_urls = wrap.intraday_url(settings, function, ticker_lst, interval, outputsize = outputsize, output = "lst")

s = time.time()

with concurrent.futures.ThreadPoolExecutor() as executor: 
    futures = []
    data = []
    for url in test_urls: 
        futures.append(executor.submit(wrap.get_csv_data, url = url))
    for future in concurrent.futures.as_completed(futures): 
        data.append(future.result())

e = time.time()
print(f'Total time elapsed: {e-s} seconds')

s = time.time()
result2 = []
for i in test_urls: 
    result2.append(wrap.get_csv_data(i))
e = time.time()
print(f'Total time elapsed: {e-s} seconds')

Total time elapsed: 10.854691505432129 seconds
Total time elapsed: 144.65774774551392 seconds


In [15]:
print(test_urls[0])
print(len(data[4]))

https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=CYAN&interval=1min&outputsize=compact&datatype=csv&apikey=MD27QTTVAK3AJBUQ
612


2761.3333333333335 92.04444444444445 11.505555555555556
